## Directory Structure

### Source Jobs (Uploads)

```
data/sources/job_metadata/
  2026-01-01_20-48-14.json              # What was uploaded
  latest.json -> 2026-01-01_20-48-14.json
```

### Analytics Jobs (Processing)

```
data/analytics/
  
  job_registry/                          # Central hub: full pipeline trace
    2026-01-01_21-37-23.json            # Complete job record with all steps
    latest.json -> 2026-01-01_...json
  
  ocr/
    job_metadata/                        # OCR-specific metadata
      2026-01-01_21-37-23.json          # OCR configs, results summary
    citekey1/
      2026-01-01_21-37-23/
        citekey1.json                    # OCR results for this citekey
    latest -> 2026-01-01_21-37-23
  
  toc/
    job_metadata/
      2026-01-01_21-37-24.json
    citekey1/
      2026-01-01_21-37-24/
        citekey1_toc.json
  
  segmentation/
    job_metadata/
      2026-01-01_21-37-25.json
    citekey1/
      2026-01-01_21-37-25/
        citekey1_segments.json
```

### File Naming

- **`job_registry/`** = Central registry (hub) — what ran in the entire pipeline
- **`{step}/job_metadata/`** = Task-specific metadata (spokes) — details for one step
- **Same job ID everywhere** = Easy to link files across pipeline

## Central Registry Format

The central registry (`data/analytics/job_registry/{job_id}.json`) contains the complete story of a job:

```json
{
  "job_id": "2026-01-01_21-37-23",
  "timestamp": "2026-01-01T21:37:23Z",
  
  "source": {
    "job_id": "2026-01-01_20-48-14",
    "bucket": "cna-sources",
    "citekeys": {
      "total": 50,
      "list": ["citekey1", "citekey2", ...]
    },
    "checksums": {
      "citekey1": "abc123..."
    }
  },
  
  "pipeline_steps": {
    "ocr": {
      "job_id": "2026-01-01_21-37-23",
      "status": "completed",
      "timestamp": "2026-01-01T21:37:23Z",
      "metadata_path": "ocr/job_metadata/2026-01-01_21-37-23.json",
      "citekeys": {
        "total": 50,
        "successful": 50,
        "failed": 0,
        "list": [...]
      },
      "results_summary": {
        "total_pages": 5000,
        "total_images": 0
      }
    },
    "toc": {
      "job_id": null,
      "status": "pending"
    },
    "segmentation": {
      "job_id": null,
      "status": "pending"
    }
  },
  
  "execution_trace": [
    {
      "step": "ocr",
      "job_id": "2026-01-01_21-37-23",
      "timestamp": "2026-01-01T21:37:23Z",
      "status": "completed"
    }
  ]
}
```

**Key Fields:**

- **`source`** — Links back to original upload (with checksums!)
- **`pipeline_steps`** — Shows status of each planned step
- **`execution_trace`** — Chronological record of what actually ran

## Task-Specific Metadata Format

Each step's metadata (`data/analytics/{step}/job_metadata/{job_id}.json`) contains task-specific details:

```json
{
  "status": "completed",
  "job_id": "2026-01-01_21-37-23",
  "job_timestamp": "2026-01-01T21:37:23Z",
  "pipeline_step": "ocr",
  
  "source_job_id": "2026-01-01_20-48-14",
  
  "citekeys": {
    "total": 50,
    "successful": 50,
    "failed": 0,
    "list": [...]
  },
  
  "results_summary": {
    "total_pages": 5000,
    "total_images": 0
  },
  
  "config": {
    "pipeline_config": { ... },
    "predict_params": { ... }
  },
  
  "batching": {
    "num_batches": 2,
    "pages_per_batch": 300
  },
  
  "runpod_jobs": {
    "runpod_job_id_1": ["citekey1", "citekey2"],
    "runpod_job_id_2": ["citekey3", "citekey4"]
  }
}
```

## Querying Jobs

Use the `etl_metadata` module to query jobs programmatically.

### List All Jobs

```python
from scripts.etl_metadata import list_all_jobs, get_latest_job

# All jobs from central registry (all steps)
all_jobs = list_all_jobs()
# Output: ["2026-01-01_21-37-23", "2026-01-01_20-48-14", ...]

# Jobs from a specific step only
ocr_jobs = list_all_jobs(step_name="ocr")
# Output: ["2026-01-01_21-37-23", "2026-01-01_20-20-00", ...]

# Get latest job
latest = get_latest_job()
# Output: "2026-01-01_21-37-23"

latest_ocr = get_latest_job(step_name="ocr")
# Output: "2026-01-01_21-37-23"
```

### Get Job Details

```python
from scripts.etl_metadata import get_central_registry, get_step_metadata

# Get full pipeline trace
registry = get_central_registry("2026-01-01_21-37-23")
# Shows which steps completed, source job, execution trace

# Get step-specific metadata
ocr_metadata = get_step_metadata("ocr", "2026-01-01_21-37-23")
# Shows OCR configs, results summary, citekeys processed
```

### Check Pipeline Status

```python
from scripts.etl_metadata import get_job_pipeline_status

status = get_job_pipeline_status("2026-01-01_21-37-23")
# Output:
# {
#   "job_id": "2026-01-01_21-37-23",
#   "timestamp": "2026-01-01T21:37:23Z",
#   "steps_completed": ["ocr"],
#   "steps_pending": ["toc", "segmentation"],
#   "source_citekeys": ["citekey1", "citekey2", ...],
#   "source_job_id": "2026-01-01_20-48-14"
# }
```

### Find Jobs for a Citekey

```python
from scripts.etl_metadata import find_jobs_for_citekey

# Find all jobs that processed a specific citekey
jobs = find_jobs_for_citekey("citekey1")
# Output:
# [
#   {"job_id": "2026-01-01_20-48-14", "step": "source (upload)", "status": "completed"},
#   {"job_id": "2026-01-01_21-37-23", "step": "ocr", "status": "completed"},
#   {"job_id": "2026-01-01_21-37-24", "step": "toc", "status": "completed"}
# ]
```

## Common Workflows

### Workflow 1: Check What Was Uploaded

```python
from scripts.etl_metadata import get_central_registry

# Get a job's source information
registry = get_central_registry("2026-01-01_21-37-23")

# See which PDFs were uploaded
source = registry["source"]
print(f"Uploaded from job: {source['job_id']}")
print(f"Citekeys: {source['citekeys']['list']}")
print(f"Bucket: {source['bucket']}")

# Check checksums to verify file integrity
checksums = source['checksums']
print(f"SHA256 of citekey1: {checksums['citekey1']}")
```

### Workflow 2: Verify OCR Completed Successfully

```python
from scripts.etl_metadata import get_job_pipeline_status

status = get_job_pipeline_status("2026-01-01_21-37-23")

if "ocr" in status["steps_completed"]:
    print("✓ OCR completed")
else:
    print("✗ OCR not yet completed")

# Get OCR results summary
registry = get_central_registry("2026-01-01_21-37-23")
ocr_step = registry["pipeline_steps"]["ocr"]
print(f"Pages processed: {ocr_step['results_summary']['total_pages']}")
print(f"Successful citekeys: {ocr_step['citekeys']['successful']}")
```

### Workflow 3: Trace a Citekey Through the Pipeline

```python
from scripts.etl_metadata import find_jobs_for_citekey

citekey = "citekey1"
jobs = find_jobs_for_citekey(citekey)

print(f"Complete lineage for {citekey}:")
for job in jobs:
    print(f"  {job['step']:<20} | Job: {job['job_id']} | Status: {job['status']}")

# Output:
#   source (upload)      | Job: 2026-01-01_20-48-14 | Status: completed
#   ocr                  | Job: 2026-01-01_21-37-23 | Status: completed
#   toc                  | Job: 2026-01-01_21-37-24 | Status: completed
```

### Workflow 4: Find Latest OCR Job and Its Results

```python
from scripts.etl_metadata import get_latest_job, get_step_metadata
from pathlib import Path

# Get latest OCR job
job_id = get_latest_job(step_name="ocr")
print(f"Latest OCR job: {job_id}")

# Load OCR results
metadata = get_step_metadata("ocr", job_id)
print(f"Citekeys processed: {metadata['citekeys']['list']}")
print(f"Total pages: {metadata['results_summary']['total_pages']}")

# Access OCR output files
results_dir = Path("data/analytics/ocr") / metadata['citekeys']['list'][0] / job_id
print(f"Results directory: {results_dir}")
```

### Workflow 5: Rebuild Central Registry (if corrupted)

```python
from scripts.etl_metadata import rebuild_central_registry

# Scan all step directories and rebuild central registry
job_count = rebuild_central_registry()
print(f"Rebuilt registry with {job_count} jobs")
```

## Understanding Lineage

### Complete Pipeline Trace

```
Source PDF (B2)
        ↓
   Upload Job (2026-01-01_20-48-14)
        ↓ source_job_id
   OCR Job (2026-01-01_21-37-23)
        ↓
   TOC Extraction Job (2026-01-01_21-37-24)
        ↓
   Segmentation Job (2026-01-01_21-37-25)
```

### How Lineage is Stored

**Central Registry (`data/analytics/job_registry/2026-01-01_21-37-23.json`):**

```json
{
  "source": {
    "job_id": "2026-01-01_20-48-14",
    "citekeys": { ... }
  },
  "pipeline_steps": {
    "ocr": {
      "status": "completed",
      "citekeys": { ... }
    }
  }
}
```

One file shows:
- ✅ Which PDFs were uploaded (from source)
- ✅ Which PDFs were processed by OCR
- ✅ Which PDFs will be processed by next steps
- ✅ Configuration used at each step
- ✅ Timestamps of when each step ran

### Why This Matters

**Reproducibility**: "Exactly which PDF was used to produce this OCR?"
→ Check source job ID and checksums

**Debugging**: "If OCR output is wrong, where did the PDF come from?"
→ Follow the source_job_id back to the upload

**Audit Trail**: "What changed between OCR runs?"
→ Compare configs in two job_registry files

**Data Integrity**: "Have these files been tampered with?"
→ Compare stored checksums with B2 files

## Best Practices

### For Data Workers

1. **Always check the central registry** to understand what happened:
   ```bash
   cat data/analytics/job_registry/latest.json | python -m json.tool
   ```

2. **Note job IDs when you run pipelines**:
   ```
   Uploaded on 2026-01-01 with job: 2026-01-01_20-48-14
   Processed OCR on 2026-01-01 with job: 2026-01-01_21-37-23
   ```

3. **Use job tracking for documentation**:
   - Document which job processed which batch
   - Note any issues and their job IDs
   - Keep job IDs in processing notes

### For Developers

1. **Check job registry for debugging**:
   - See which steps have completed
   - Compare configs between different job runs
   - Identify bottlenecks in the pipeline

2. **Use metadata for monitoring**:
   ```python
   from scripts.etl_metadata import get_job_pipeline_status
   
   # Build a monitoring dashboard
   status = get_job_pipeline_status(latest_job_id)
   print(f"Completed: {len(status['steps_completed'])} steps")
   print(f"Pending: {len(status['steps_pending'])} steps")
   ```

3. **Never delete job metadata**:
   - It's your audit trail
   - Enables reproducibility
   - Costs nothing to keep

### For the Pipeline

1. **Lineage is automatic** — each step automatically links to source
2. **Central registry is self-contained** — no need to cross-reference files
3. **Checksums enable verification** — compare stored vs actual file hashes
4. **Timestamps enable ordering** — understand the sequence of operations

## Troubleshooting

### Job Registry Entry Missing

**Problem**: Can't find `data/analytics/job_registry/{job_id}.json`

**Possible Causes**:
1. Job ID is from a source upload (check `data/sources/job_metadata/` instead)
2
,
3
,
,

### Central Registry Out of Sync

**Problem**: Central registry is missing entries from step metadata directories

**Solution**: Rebuild the registry
```python
from scripts.etl_metadata import rebuild_central_registry
rebuild_central_registry()
```

### Can't Find Citekey in Pipeline

**Problem**: Can't trace a citekey through OCR and TOC steps

**Solution**: Use the lineage finder
```python
from scripts.etl_metadata import find_jobs_for_citekey
jobs = find_jobs_for_citekey("my_citekey")
print(jobs)
```

This will show all jobs that touched your citekey, including the source upload.

## Summary

| Aspect | Details |
|--------|----------|
| **Central Registry** | `data/analytics/job_registry/{job_id}.json` — Single source of truth |
| **Task Metadata** | `data/analytics/{step}/job_metadata/{job_id}.json` — Step-specific details |
| **Job ID** | Timestamp format: `YYYY-MM-DD_HH-MM-SS` — Globally unique |
| **Lineage** | Source → Upload → OCR → TOC → Segmentation |
| **Key Link** | `source_job_id` in metadata links analytics jobs back to uploads |
| **Query API** | Use `etl_metadata` module for programmatic access |
| **Reproducibility** | Checksums + timestamps enable exact recreation of any job |

**Bottom Line**: Every job generates a complete audit trail. Use job IDs to trace data through the entire pipeline and verify that everything ran correctly.